In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
df = pd.read_csv("data_pepTestCustomers.csv")
new = pd.read_csv("data_pepNewCustomers.csv")

In [11]:
mdf = df.copy()
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
id             600 non-null object
age            540 non-null float64
sex            600 non-null int64
region         600 non-null int64
income         600 non-null float64
married        600 non-null int64
children       600 non-null int64
car            600 non-null int64
save_act       600 non-null int64
current_act    600 non-null int64
mortgage       600 non-null int64
pep            600 non-null int64
dtypes: float64(2), int64(9), object(1)
memory usage: 56.3+ KB


In [12]:
mdf.age.fillna(round(mdf.age.mean(),0), inplace=True)
mdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
id             600 non-null object
age            600 non-null float64
sex            600 non-null int64
region         600 non-null int64
income         600 non-null float64
married        600 non-null int64
children       600 non-null int64
car            600 non-null int64
save_act       600 non-null int64
current_act    600 non-null int64
mortgage       600 non-null int64
pep            600 non-null int64
dtypes: float64(2), int64(9), object(1)
memory usage: 56.3+ KB


In [13]:
mdf['realincome'] = np.where(mdf['children']==0, mdf['income'], mdf['income']/mdf['children'])

In [14]:
columns = ['income', 'children', 'current_act', 'car', 'mortgage', 'region']
mdf = mdf.drop(columns, axis=1)
mdf.head()

,id,age,sex,married,save_act,pep,realincome
0,ID12101,48.0,0,0,0,1,17546.000000
1,ID12102,40.0,1,1,0,0,10028.366667
2,ID12103,43.0,0,1,1,0,16575.400000
3,ID12104,23.0,0,1,0,0,6791.800000
4,ID12105,57.0,0,1,1,0,50576.300000


In [15]:
from sklearn.model_selection import train_test_split  

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.neural_network import MLPClassifier 

In [18]:
dfX = mdf.drop(['id','pep'], axis=1)  
dfy = mdf['pep']                    
X_train, X_test, y_train, y_test = train_test_split(dfX, dfy, test_size=0.25, random_state=0)

In [19]:
tree = DecisionTreeClassifier(max_depth=6, random_state=0)
tree.fit(X_train, y_train)
display(tree.score(X_test, y_test))

0.66

In [20]:
svm = SVC(random_state=0)
svm.fit(X_train, y_train)
display(svm.score(X_test, y_test))

c:\users\kduar_000\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.5266666666666666

In [21]:
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
display(mlp.score(X_test, y_test))

0.48

In [22]:
best_model = tree

In [23]:
ndf = new.copy()
ndf['realincome'] = np.where(ndf['children']==0, ndf['income'], ndf['income']/ndf['children'])
ndf = ndf.drop(columns, axis=1)
ndf.head()

,id,age,sex,married,save_act,realincome
0,ID12701,23,1,1,1,18766.90
1,ID12702,30,1,0,1,9915.67
2,ID12703,45,0,0,1,21881.60
3,ID12704,50,1,1,1,23397.20
4,ID12705,41,0,1,1,20721.10


In [24]:
ndf['pred'] = best_model.predict(ndf.loc[:,'age':'realincome'])
ndf['pred_prob'] = best_model.predict_proba(ndf.loc[:,'age':'realincome'])[:,1]
ndf.head()

,id,age,sex,married,save_act,realincome,pred,pred_prob
0,ID12701,23,1,1,1,18766.90,0,0.485294
1,ID12702,30,1,0,1,9915.67,0,0.000000
2,ID12703,45,0,0,1,21881.60,1,1.000000
3,ID12704,50,1,1,1,23397.20,1,1.000000
4,ID12705,41,0,1,1,20721.10,0,0.485294


In [25]:
target = ndf.query('pred == 1 & pred_prob > 0.7')  # PEP에 가입할 확율이 70%가 넘는 고객만 추출
target.sort_values(by="pred_prob", ascending=False).to_csv("pep_target.csv", index=False)
pd.read_csv("pep_target.csv").tail()

,id,age,sex,married,save_act,realincome,pred,pred_prob
56,ID12869,41,1,0,1,21122.6,1,0.866667
57,ID12798,37,1,0,1,31979.9,1,0.866667
58,ID12755,34,1,0,1,17221.3,1,0.866667
59,ID12706,20,1,0,1,16688.5,1,0.866667
60,ID12782,44,1,0,1,39202.2,1,0.866667
